In [46]:
import setup
setup.init_django()

In [47]:
from market.models import StockQuote

In [48]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [49]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

# filtering data
qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

4340

In [50]:
total = 0
for obj in qs:
    total += obj.close_price

In [51]:
total /qs.count()

Decimal('219.3371326958525345622119816')

In [52]:
# average prices for entire month
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 2, 'avg_price': Decimal('170.1696036020583190')}, {'company': 1, 'avg_price': Decimal('226.9668025019962736')}]>

In [53]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker = ticker).order_by('time')[:count]) # grabbing orginal window
rolling_qs.reverse() # orginal time order

In [54]:
rolling_qs

[<StockQuote: StockQuote object (313486)>,
 <StockQuote: StockQuote object (313485)>,
 <StockQuote: StockQuote object (313484)>,
 <StockQuote: StockQuote object (313483)>,
 <StockQuote: StockQuote object (313482)>]

In [55]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 313486 234.4180 234.4180
2 313485 234.5550 234.4865
3 313484 234.6300 234.5343333333333333333333333
4 313483 234.4250 234.5070
5 313482 234.5300 234.5116


In [56]:
qs = StockQuote.objects.filter(company__ticker = ticker, time__range=(start_date, end_date))

In [62]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma = Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=frame_start, end=0), # based on amount we want to get
        )
).order_by('-time')

In [64]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma)

330238 235.5300 235.5260000000000000
330237 235.3600 235.5400000000000000
330236 235.4900 235.5833600000000000
330235 235.6000 235.6113600000000000
330234 235.6500 235.6153600000000000


In [65]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5 = Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=-4, end=0), # based on amount we want to get
        ),
        ma_20 = Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=-19, end=0), # based on amount we want to get
        )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2352431156.py, line 3)